In [ ]:
# mount your Google Drive
from google.colab import drive
drive.mount("/content/drive")

base_dir = './data'

Mounted at /content/drive


## Try to crawl instructional videos (starts with how to ...) from youtube

In [ ]:
!pip install youtube-search-python
!pip install yt_dlp
!pip install youtube-transcript-api
!pip install webvtt-py

In [ ]:
import googleapiclient.discovery
import pandas as pd
import re
import webvtt
import os

In [ ]:
api_key = "XXX"
keyword = "how to make"

youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)
request = youtube.search().list(q=keyword, part="snippet", type="video", maxResults=50, videoCaption="closedCaption", videoSyndicated="true")

In [ ]:
def duration_iso8601_to_mmss(duration_iso8601):
    hours = re.search('(\d+)H', duration_iso8601)
    minutes = re.search('(\d+)M', duration_iso8601)
    seconds = re.search('(\d+)S', duration_iso8601)

    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    seconds = int(seconds.group(1)) if seconds else 0

    total_seconds = hours * 3600 + minutes * 60 + seconds
    duration_mmss = "{:02}:{:02}".format(total_seconds // 60, total_seconds % 60)
    return duration_mmss

In [ ]:
response = request.execute()
df = pd.DataFrame(columns=["name", "url", "duration", "id"])
count = 0
while count < 200 and request is not None:
    video_ids = [item["id"]["videoId"] for item in response["items"]]

    details_request = youtube.videos().list(id=",".join(video_ids), part="contentDetails")
    details_response = details_request.execute()

    for i, item in enumerate(response["items"]):
        title = item["snippet"]["title"]
        video_id = item["id"]["videoId"]
        url = "https://www.youtube.com/watch?v=" + video_id

        duration_iso8601 = details_response["items"][i]["contentDetails"]["duration"]
        duration = duration_iso8601_to_mmss(duration_iso8601)

        print(duration, title, url)

        df.loc[count] = [title, url, duration, video_id]

        count += 1
    request = youtube.search().list_next(request, response)
    if request is not None:
        response = request.execute()

# save the dataframe as a csv file named raw_youtube_videos.csv
df.to_csv("raw_youtube_videos.csv", index=False)

07:49 How To Make a Paper Ninja Star (Shuriken) - Origami https://www.youtube.com/watch?v=ktLR-WjB_C8
08:11 How to make ONLY PAPER diy MINI NOTEBOOK. Easy origami notebook no glue no scissors. https://www.youtube.com/watch?v=KQxBvI4Nt10
10:15 How To Make Yourself FLOAT! https://www.youtube.com/watch?v=f7r6PeUjNEE
07:16 How To Make Birria Tacos https://www.youtube.com/watch?v=gCSEXjm6B-Y
04:23 The 7 Things You Can Do to Make a Relationship Work https://www.youtube.com/watch?v=hjyuzRwMK1M
05:00 Chili Recipe - How to Make Homemade Chili https://www.youtube.com/watch?v=9g-gMkV5vL4
12:04 How To Make Hello Kitty House With Rainbow Slide Pool From Cardboard ❤️ DIY Miniature House #5 https://www.youtube.com/watch?v=kZwJRamGjWs
02:10 How to Make Meatballs https://www.youtube.com/watch?v=6ViUx0xui8Q
05:54 How To Make Sugar Rockets https://www.youtube.com/watch?v=12fR9neVnS8
05:15 How to Make Magnetic Slime - Science Experiment https://www.youtube.com/watch?v=bZ1WmeKir78
20:22 How to make a Log C

In [ ]:
import yt_dlp

video_urls = []
for index, row in df.iterrows():
    video_urls.append(row["url"])

ydl_opts = {
    'writesubtitles': True,
    'subtitlesformat': 'vtt',
    'subtitleslangs': ['en'], # choose subtitle language (en for English)
    'outtmpl': 'subs/%(id)s.%(ext)s',
    'skip_download': True,
}

ydl = yt_dlp.YoutubeDL(ydl_opts)
ydl.download(video_urls)

[youtube] Extracting URL: https://www.youtube.com/watch?v=ktLR-WjB_C8
[youtube] ktLR-WjB_C8: Downloading webpage
[youtube] ktLR-WjB_C8: Downloading android player API JSON
[info] ktLR-WjB_C8: Downloading subtitles: en
[info] ktLR-WjB_C8: Downloading 1 format(s): 247+251
[info] Writing video subtitles to: subs/ktLR-WjB_C8.en.vtt
[download] Destination: subs/ktLR-WjB_C8.en.vtt
[download] 100% of    1.67KiB in 00:00:00 at 52.45KiB/s
[youtube] Extracting URL: https://www.youtube.com/watch?v=KQxBvI4Nt10
[youtube] KQxBvI4Nt10: Downloading webpage
[youtube] KQxBvI4Nt10: Downloading android player API JSON
[youtube] KQxBvI4Nt10: Downloading player 21246a91
[info] KQxBvI4Nt10: Downloading subtitles: en
[info] KQxBvI4Nt10: Downloading 1 format(s): 315+251
[info] Writing video subtitles to: subs/KQxBvI4Nt10.en.vtt
[download] Destination: subs/KQxBvI4Nt10.en.vtt
[download] 100% of    486.00B in 00:00:00 at 19.63KiB/s
[youtube] Extracting URL: https://www.youtube.com/watch?v=f7r6PeUjNEE
[youtube] f

0

In [ ]:
labels = []
videos_df = pd.DataFrame(columns=["name", "url", "duration", "id"])
count = 0

for index, row in df.iterrows():
    flag = False
    video_id = row["id"]
    video_url = row["url"]

    file_path = f"subs/{video_id}.en.vtt"
    captions = []
    if os.path.exists(file_path):
        captions = webvtt.read(file_path)

    for caption in captions:
        text = caption.text.strip()
        duration = caption.end_in_seconds - caption.start_in_seconds

        if len(text) > 20:
            labels.append((video_id, video_url, text, duration, caption.start_in_seconds, caption.end_in_seconds))
            flag = True

    if flag:
        videos_df.loc[count] = [row["name"], row["url"], row["duration"], row["id"]]
        count += 1

labels_df = pd.DataFrame(labels, columns=["id", "video_url", "text", "duration", "start", "end"])

labels_df.to_csv("labels.csv", index=False)
videos_df.to_csv("youtube_videos.csv", index=False)

# **Data collection and automatic labeling for dense video captioning models**

### imports and settings

In [ ]:
BRANCH = 'main'
!pip install ijson
!pip install isodate
!pip install remotezip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for remotezip: filename=remotezip-0.12.1-py3-none-any.whl size=7934 sha256=230cce71544503d2938968c74bbed6021cb33d35fb9e3fc94ed08ae9ebf91bb2
  Stored in directory: /root/.cache/pip/wheels/fc/76/04/beed1a6df4eb7430ee13c3900746edd517e5e597298d1f73f3
Successfully built remotezip


In [ ]:
import pandas as pd
import ijson
from tqdm import tqdm
import ast
import os
import re
import csv
import gc
import requests
import json
import datetime
import isodate
import remotezip as rz
from urllib.parse import urlparse


import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Nemo

In [ ]:
!pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

from nemo.collections.nlp.models import PunctuationCapitalizationModel

# to get the list of pre-trained models
PunctuationCapitalizationModel.list_available_models()

# Download and load the pre-trained BERT-based model
punctuation_model = PunctuationCapitalizationModel.from_pretrained("punctuation_en_bert")

## 1. Extract key moments from video descriptions using Youtube api

### extract cooking videos list

In [ ]:
ht100mv1 = pd.read_csv(base_dir + 'HowTo100M_v1.csv', index_col=0)
cooking_cat2_list = ['Recipes', 'Food Preparation', 'Holiday Cooking', 'Cooking for Children']
video_ids_list = list(ht100mv1[ht100mv1.category_2.isin(cooking_cat2_list)].index)

df = pd.DataFrame(video_ids_list, columns=['keys'])
df.to_csv(base_dir + 'cooking_videos.csv')

### key moments

In [ ]:
api_key = "XXX"

In [ ]:
def extract_chapters_from_description(description, duration):
    chapters = []
    lines = description.split("\n")
    for line in lines:
        re_time = re.search(r"(\d{2}:)?\d{2}:\d{2}", line)
        if re_time:
            time = re_time.group()
            before = line[:re_time.start()].strip()
            after = line[re_time.end():].strip()

            if before.endswith("-"):
                before = before[:-1].strip()
            if after.startswith("-"):
                after = after[1:].strip()

            title = before + "." + after
            title = title.strip('.')
            chapters.append((title, time))

    if len(chapters) < 3:
        return []
    # if chapters[0][1] != "00:00" or chapters[0][1] != "00:00:00":
    #     return []

    prev_time = datetime.timedelta(seconds=sum(int(x) * 60 ** i for i, x in enumerate(reversed(chapters[0][1].split(":")))))
    valid_chapters = [(chapters[0][0], prev_time)]
    for chapter in chapters:
        curr_time_str = chapter[1]
        curr_time = datetime.timedelta(seconds=sum(int(x) * 60 ** i for i, x in enumerate(reversed(curr_time_str.split(":")))))
        if curr_time > prev_time and curr_time.total_seconds() < duration:
            valid_chapters.append((chapter[0], curr_time))
            prev_time = curr_time

    return valid_chapters


def get_chapters(video_id):
    response = requests.get(f"https://www.googleapis.com/youtube/v3/videos?part=snippet,contentDetails&id={video_id}&key={api_key}")
    data = response.json()
    print(data)

    try:
        description = data["items"][0]["snippet"]["description"]
        description = description.replace("\r", "")
        duration = data["items"][0]["contentDetails"]["duration"]
    except Exception as e:
        return []
    duration = isodate.parse_duration(duration).total_seconds()

    chapters = extract_chapters_from_description(description, duration)
    final_chapters = []
    for chapter in chapters:
        final_chapters.append({'title': chapter[0], 'time': chapter[1]})

    return final_chapters

In [ ]:
data_with_key_moment = {'id': [], 'chapters': []}
for i in tqdm(range(len(df['keys']))):
    key = df['keys'][i]
    chapters = get_chapters(key)

    if chapters:
        data_with_key_moment['id'].append(key)
        data_with_key_moment['chapters'].append(chapters)

    if i % 500 == 499:
        df_dwkm = pd.DataFrame.from_dict(data_with_key_moment)
        df_dwkm.to_csv(base_dir + 'data_with_key_moment.csv',
                       mode='a', header=not os.path.exists(base_dir + 'data_with_key_moment.csv'), index=False)
        data_with_key_moment = {'id': [], 'chapters': []}

In [ ]:
df_dwkm = pd.DataFrame.from_dict(data_with_key_moment)
df_dwkm.to_csv(base_dir + 'data_with_key_moment.csv')

## 2. Punctuate raw video transcripts and extract most important sentences using action verbs from it

### Extract time of each word

In [ ]:
!unzip $base_dir"raw_caption_superclean.zip"

Archive:  drive/MyDrive/University/01022-Final-Project/raw_caption_superclean.zip
replace raw_caption_superclean.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
data = ijson.parse(open('raw_caption_superclean.json', 'r'))

BATCH = 1000

current_key = ''
current_data = {'text': [], 'start': [], 'end': []}
processed_data = {'id': [], 'text': [], 'times': []}

cooking_videos = list(pd.read_csv(base_dir + 'cooking_videos.csv')['keys'])

In [ ]:
def calculate_time(processed_data, key, data):
    text = ''
    times = []
    for i, sentence in enumerate(data['text']):
        split_sentence = sentence.split()
        start = data['start'][i]
        end = data['end'][i]
        for j, word in enumerate(split_sentence):
            times.append(round(start + ((end - start) / len(split_sentence)) * j, 2))
        if i != 0:
            text += ' '
        text += sentence
    processed_data['id'].append(key)
    processed_data['text'].append(text)
    processed_data['times'].append(times)

In [ ]:
len(cooking_videos)

376076

In [ ]:
it = 1
with tqdm(total=376076) as progress_bar:
    for prefix, event, value in data:
        if prefix == '':
            if current_key != '':

                if current_key in cooking_videos:
                    calculate_time(processed_data, current_key, current_data)
                    it += 1
                    progress_bar.update()

                    if it % BATCH == 0:
                        df = pd.DataFrame.from_dict(processed_data)
                        df.to_csv('time_extracted_data.csv', mode='a',
                                  header=not os.path.exists('time_extracted_data.csv'), index=False)

                        processed_data = {'id': [], 'text': [], 'times': []}

            current_key = ''
            current_data = {'text': [], 'start': [], 'end': []}
            continue
        else:
            if current_key == '':
                current_key = prefix
            if prefix.endswith('.text.item'):
                current_data['text'].append(value)
            elif prefix.endswith('.start.item'):
                current_data['start'].append(value)
            elif prefix.endswith('.end.item'):
                current_data['end'].append(value)

In [ ]:
!zip $base_dir"time_extracted_data.zip" "time_extracted_data.csv"

### Add punctuation

In [ ]:
!unzip $base_dir"time_extracted_data.zip"

Archive:  drive/MyDrive/University/01022-Final-Project/time_extracted_data.zip
  inflating: time_extracted_data.csv  


In [ ]:
it = 0
for chunk in pd.read_csv('time_extracted_data.csv', chunksize=1000):
    # if it < 2:
    #     it += 1
    #     continue
    chunk['punctuated_text'] = punctuation_model.add_punctuation_capitalization(chunk['text'], batch_size=50)
    chunk.to_csv('punctuated_data.csv', mode='a',
              header=not os.path.exists('punctuated_data.csv'), index=False)
    gc.collect()
    it += 1
    # if it == 10:
    #     break

[NeMo I 2023-09-06 13:39:28 punctuation_capitalization_infer_dataset:127] Max length: 64
[NeMo I 2023-09-06 13:39:28 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-09-06 13:39:28 data_preprocessing:406] Min: 100 |                  Max: 3252 |                  Mean: 726.399 |                  Median: 642.5
[NeMo I 2023-09-06 13:39:28 data_preprocessing:412] 75 percentile: 943.00
[NeMo I 2023-09-06 13:39:28 data_preprocessing:413] 99 percentile: 2102.21


100%|██████████| 1690/1690 [05:35<00:00,  5.03batch/s]


[NeMo I 2023-09-06 13:46:38 punctuation_capitalization_infer_dataset:127] Max length: 64
[NeMo I 2023-09-06 13:46:38 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-09-06 13:46:38 data_preprocessing:406] Min: 104 |                  Max: 2571 |                  Mean: 726.047 |                  Median: 634.0
[NeMo I 2023-09-06 13:46:38 data_preprocessing:412] 75 percentile: 971.25
[NeMo I 2023-09-06 13:46:38 data_preprocessing:413] 99 percentile: 1983.26


100%|██████████| 1689/1689 [05:34<00:00,  5.05batch/s]


[NeMo I 2023-09-06 13:53:42 punctuation_capitalization_infer_dataset:127] Max length: 64
[NeMo I 2023-09-06 13:53:42 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-09-06 13:53:42 data_preprocessing:406] Min: 88 |                  Max: 2351 |                  Mean: 655.271 |                  Median: 561.5
[NeMo I 2023-09-06 13:53:42 data_preprocessing:412] 75 percentile: 893.50
[NeMo I 2023-09-06 13:53:42 data_preprocessing:413] 99 percentile: 1878.01


...

100%|██████████| 1710/1710 [05:38<00:00,  5.05batch/s]


[NeMo I 2023-09-06 14:22:56 punctuation_capitalization_infer_dataset:127] Max length: 64
[NeMo I 2023-09-06 14:22:56 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-09-06 14:22:56 data_preprocessing:406] Min: 102 |                  Max: 5037 |                  Mean: 795.247 |                  Median: 664.5
[NeMo I 2023-09-06 14:22:56 data_preprocessing:412] 75 percentile: 1042.25
[NeMo I 2023-09-06 14:22:56 data_preprocessing:413] 99 percentile: 2469.58


100%|██████████| 1863/1863 [06:08<00:00,  5.06batch/s]


[NeMo I 2023-09-06 14:30:37 punctuation_capitalization_infer_dataset:127] Max length: 64
[NeMo I 2023-09-06 14:30:37 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-09-06 14:30:37 data_preprocessing:406] Min: 106 |                  Max: 2216 |                  Mean: 666.621 |                  Median: 579.5
[NeMo I 2023-09-06 14:30:37 data_preprocessing:412] 75 percentile: 883.75
[NeMo I 2023-09-06 14:30:37 data_preprocessing:413] 99 percentile: 1868.30


100%|██████████| 1541/1541 [05:04<00:00,  5.07batch/s]


In [ ]:
!zip $base_dir"punctuated_data.zip" "punctuated_data.csv"

updating: punctuated_data.csv (deflated 79%)


### Select sentences with action verbs

In [ ]:
!unzip $base_dir"punctuated_data.zip"

Archive:  drive/MyDrive/University/01022-Final-Project/punctuated_data.zip
  inflating: punctuated_data.csv     


In [ ]:
task_id_path = base_dir + 'task_ids.csv'

def get_action_verbs(path):
    action_verbs = []
    with open(path, 'r') as f:
        csvreader = csv.reader(f)

        for row in csvreader:
            res = re.sub(r'\d', '', row[0])
            res = re.sub('\t', '', res)
            res = ' '.join(res.split(' '))

            action_verbs.append(res.lower())

    return action_verbs


action_verbs = get_action_verbs(task_id_path)
action_verbs = set(action_verbs)

In [ ]:
with tqdm() as progress_bar:
    it = 0
    for data_chunk in pd.read_csv('punctuated_data.csv', chunksize=1000):
        # if it < 2:
        #     it += 1000
        #     continue
        for text in data_chunk['punctuated_text']:
            sentences = nltk.sent_tokenize(text)

            key = data_chunk['id'][it]
            times_str = data_chunk['times'][it]
            times = times_str.strip("[]").split(", ")
            times = [float(x.strip("Decimal('')")) for x in times]
            times.sort()

            time_index = 0
            processed_data = {'id': [], 'start': [], 'end': [], 'sentence': [], 'verb': []}
            for sentence in sentences:
                try:
                    start_time = times[time_index]
                except Exception:
                    break

                time_index += len(sentence.split())

                try:
                    end_time = times[time_index - 1]
                except Exception:
                    end_time = start_time + 4

                words = nltk.word_tokenize(sentence)
                pos_tags = nltk.pos_tag(words)

                prv_word = ''
                prv_2_word = ''
                for word, pos in pos_tags:
                    word_2 = prv_word + ' ' + word
                    word_3 = prv_2_word + ' ' + word_2
                    if pos.startswith('VB') and (word.lower() in action_verbs):
                        processed_data['id'].append(key)
                        processed_data['start'].append(start_time)
                        processed_data['end'].append(end_time)
                        processed_data['sentence'].append(sentence)
                        processed_data['verb'].append(word)
                        break
                    if pos.startswith('VB') and (word_2.lower() in action_verbs):
                        processed_data['id'].append(key)
                        processed_data['start'].append(start_time)
                        processed_data['end'].append(end_time)
                        processed_data['sentence'].append(sentence)
                        processed_data['verb'].append(word_2)
                        break
                    if pos.startswith('VB') and (word_3.lower() in action_verbs):
                        processed_data['id'].append(key)
                        processed_data['start'].append(start_time)
                        processed_data['end'].append(end_time)
                        processed_data['sentence'].append(sentence)
                        processed_data['verb'].append(word_3)
                        break
                    prv_2_word = prv_word
                    prv_word = word

            df = pd.DataFrame.from_dict(processed_data)
            df.to_csv('cleaned_data.csv', mode='a',
                        header=not os.path.exists('cleaned_data.csv'), index=False)
            progress_bar.update()
            it += 1

9000it [07:18, 20.51it/s]


In [ ]:
!zip $base_dir"cleaned_data.zip" "cleaned_data.csv"

updating: cleaned_data.csv (deflated 65%)


### Data cleaning

Convert data to json

In [ ]:
!unzip $base_dir"cleaned_data.zip"
df = pd.read_csv("cleaned_data.csv")

Archive:  drive/MyDrive/University/01022-Final-Project/cleaned_data.zip
replace cleaned_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cleaned_data.csv        


In [ ]:
!unzip $base_dir"punctuated_data.zip"

Archive:  drive/MyDrive/University/01022-Final-Project/punctuated_data.zip
replace punctuated_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: punctuated_data.csv     


In [ ]:
aavdc = dict()
for id in pd.read_csv("punctuated_data.csv")['id']:
    aavdc[id] = dict()

In [ ]:
for key in aavdc.keys():
    aavdc[key]['sentences'] = []
    aavdc[key]['timestamps'] = []
    aavdc[key]['verbs'] = []
    for index, row in df[df.id == key].iterrows():
        aavdc[key]['sentences'].append(row['sentence'])
        aavdc[key]['timestamps'].append([row['start'], row['end']])
        aavdc[key]['verbs'].append(row['verb'])


Normalize

In [ ]:
def normalize_clean(text):
    text = text.lower()

    words = nltk.word_tokenize(text)
    words = [word for word in words if word.isalpha()]

    # stopwords = nltk.corpus.stopwords.words('english')
    # words = [word for word in words if word not in stopwords]

    # lemmatizer = nltk.stem.WordNetLemmatizer()
    # words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

In [ ]:
for key in aavdc.keys():
    clean_sentences = []
    for sentence in aavdc[key]['sentences']:
        clean_sentences.append(normalize_clean(sentence))
    aavdc[key]['sentences'] = clean_sentences

Extract and add durations

In [ ]:
video_urls = list(pd.read_csv(base_dir + 'cooking_video_urls.csv')['keys'])
video_urls[:5]

['http://howto100m.inria.fr/dataset/mt2DcsJFqAY.mp4',
 'http://howto100m.inria.fr/dataset/nxj62Bl2rrs.mp4',
 'http://howto100m.inria.fr/dataset/7nBXRFZ67FA.mp4',
 'http://howto100m.inria.fr/dataset/5mnuzOnJk3s.mp4',
 'http://howto100m.inria.fr/dataset/HND7Ekp8fnM.mp4']

In [ ]:
video_keys = aavdc.keys()

durations = {}
with open(base_dir + 'durations.json') as json_file:
    durations = json.load(json_file)

for i in tqdm(range(10000)):
    vid = video_urls[i]
    key = video_keys[i]

    if not key in durations:
        res = !ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 -headers 'Authorization: Basic XXX' $vid
        durations[key] = float(res[0])

        if i % 500 == 499:
            with open(base_dir + "durations.json", "w") as outfile:
                json.dump(durations, outfile)

In [ ]:
with open(base_dir + "durations.json") as json_file:
    durations = json.load(json_file)

In [ ]:
for key in aavdc.keys():
    aavdc[key]['duration'] = durations[key]

Save

In [ ]:
with open(base_dir + "aavdc_10k.json", "w") as outfile:
    json.dump(aavdc, outfile)

### Extract vocabulary

In [ ]:
aavdc = json.load(open(base_dir + 'aavdc_10k.json'))

In [ ]:
def corpus_to_vocab(corpus):
    """
    Takes a corpus (list of documents) and converts
    it to two dictionaries:
      - word_to_ix: key are words in vocab, values
        are the unique indices
      - ix_to_word: key are the unique indices,
        values are the words in vocab
    """
    word_to_ix, ix_to_word = {}, {}
    ix = 0
    for document in corpus:
        for word in document:
            if word not in word_to_ix.keys():
                word_to_ix[word], ix_to_word[ix] = ix, word
                ix += 1
    return word_to_ix, ix_to_word

In [ ]:
corpus = list()
for key in aavdc.keys():
    for sentence in aavdc[key]['sentences']:
        corpus.append(sentence.split())

word_to_ix, ix_to_word = corpus_to_vocab(corpus)
vocabulary_aavdc = {'ix_to_word': ix_to_word, 'word_to_ix': word_to_ix}

In [ ]:
with open(base_dir + "vocabulary_aavdc.json", "w") as outfile:
    json.dump(vocabulary_aavdc, outfile)

### Convert data to GVL format

In [ ]:
aavdc = json.load(open(base_dir + 'aavdc_10k.json'))

In [ ]:
aavdc_train = dict(list(aavdc.items())[:8000])
aavdc_val = dict(list(aavdc.items())[8000:])

In [ ]:
with open(base_dir + "captiondata/grounding/aavdc_val_for_grounding.json", "w") as outfile:
    json.dump(aavdc_val, outfile)

In [ ]:
train_keys = list(aavdc_train.keys())
for key in train_keys:
    aavdc_train['v_' + key] = aavdc_train[key]
    del aavdc_train[key]

val_keys = list(aavdc_val.keys())
for key in val_keys:
    aavdc_val['v_' + key] = aavdc_val[key]
    del aavdc_val[key]

In [ ]:
with open(base_dir + "captiondata/aavdc_train.json", "w") as outfile:
    json.dump(aavdc_train, outfile)

with open(base_dir + "captiondata/aavdc_val.json", "w") as outfile:
    json.dump(aavdc_val, outfile)

In [ ]:
import json

p = base_dir + 'captiondata/aavdc_val.json'
out_p = base_dir + 'captiondata/para/para_aavdc_val.json'

d = json.load(open(p))
out = {}
for k,v in d.items():
    para = '. '.join(v['sentences'])
    out[k] = para
json.dump(out, open(out_p, 'w'))

## Download HowTo100M videos s3d features

In [ ]:
video_paths = list(pd.read_csv(base_dir + 'cooking_video_feature_paths.csv')['keys'])
video_paths[:5]

['howto100m_s3d_features/mt2DcsJFqAY.mp4.npy',
 'howto100m_s3d_features/nxj62Bl2rrs.mp4.npy',
 'howto100m_s3d_features/7nBXRFZ67FA.mp4.npy',
 'howto100m_s3d_features/5mnuzOnJk3s.mp4.npy',
 'howto100m_s3d_features/HND7Ekp8fnM.mp4.npy']

In [ ]:
with rz.RemoteZip('https://howto100m.inria.fr/howto100m_s3d_features/howto100m_s3d_features.zip', verify=False) as zip:
    print(zip.namelist()[0:5])
    for file_to_extract in tqdm(video_paths):
        if os.path.isfile(base_dir + 'videos/' + file_to_extract):
            continue
        try:
            zip.extract(file_to_extract, path=base_dir+'videos')
        except Exception:
            zip.extract(file_to_extract.replace(".mp4.", ".webm."), path=base_dir+'videos')

In [ ]:
import os

dir = base_dir + 'videos/howto100m_s3d_features/'

for f in os.listdir(dir):
    f_name, f_ext = os.path.splitext(f)
    f_name = f_name + f_ext
    f_name = f_name.replace(".mp4.npy", ".npy")
    f_name = f_name.replace(".webm.npy", ".npy")
    os.rename(os.path.join(dir, f), os.path.join(dir, f_name))

In [ ]:
dir = base_dir + 'videos/howto100m_s3d_features/'
count = 0
# Iterate directory
for path in os.listdir(dir):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir, path)):
        count += 1
print('File count:', count)

def get_size(start_path = dir):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

print("Size:", get_size()//1e6, 'MB')

File count: 10000
Size: 5822.0 MB
